In [17]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
import os
import time
from plotly.subplots import make_subplots
import glob

ticker_nm = '005930'
start_date  = '20190101'
today_date1 = '20231006'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

kor_stock_ohlcv_095570_total = df_raw

kor_stock_ohlcv_095570_total['MA5'] = kor_stock_ohlcv_095570_total['close'].rolling(window=5).mean()
kor_stock_ohlcv_095570_total['MA20'] = kor_stock_ohlcv_095570_total['close'].rolling(window=20).mean()
kor_stock_ohlcv_095570_total['MA60'] = kor_stock_ohlcv_095570_total['close'].rolling(window=60).mean()
kor_stock_ohlcv_095570_total['MA120'] = kor_stock_ohlcv_095570_total['close'].rolling(window=120).mean()

# MACD
kor_stock_ohlcv_095570_total['ema_short'] = kor_stock_ohlcv_095570_total['close'].rolling(window=12).mean()
kor_stock_ohlcv_095570_total['ema_long'] = kor_stock_ohlcv_095570_total['close'].rolling(window=26).mean()
kor_stock_ohlcv_095570_total['macd'] = kor_stock_ohlcv_095570_total['ema_short'] - kor_stock_ohlcv_095570_total['ema_long']


std = kor_stock_ohlcv_095570_total['close'].rolling(20).std(ddof=0)

kor_stock_ohlcv_095570_total['upper'] = kor_stock_ohlcv_095570_total['MA20'] + 2 * std
kor_stock_ohlcv_095570_total['lower'] = kor_stock_ohlcv_095570_total['MA20'] - 2 * std

kor_stock_ohlcv_095570_total['변화량'] = kor_stock_ohlcv_095570_total['close'] - kor_stock_ohlcv_095570_total['close'].shift(1)
kor_stock_ohlcv_095570_total['상승폭'] = np.where(kor_stock_ohlcv_095570_total['변화량']>=0, kor_stock_ohlcv_095570_total['변화량'], 0)
kor_stock_ohlcv_095570_total['하락폭'] = np.where(kor_stock_ohlcv_095570_total['변화량'] <0, kor_stock_ohlcv_095570_total['변화량'].abs(), 0)

# welles moving average
kor_stock_ohlcv_095570_total['AU'] = kor_stock_ohlcv_095570_total['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
kor_stock_ohlcv_095570_total['AD'] = kor_stock_ohlcv_095570_total['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
#kor_stock_ohlcv_095570_total['RS'] = kor_stock_ohlcv_095570_total['AU'] / kor_stock_ohlcv_095570_total['AD']
#kor_stock_ohlcv_095570_total['RSI'] = 100 - (100 / (1 + kor_stock_ohlcv_095570_total['RS']))
kor_stock_ohlcv_095570_total['RSI'] = kor_stock_ohlcv_095570_total['AU'] / (kor_stock_ohlcv_095570_total['AU'] + kor_stock_ohlcv_095570_total['AD']) * 100
# kor_stock_ohlcv_095570_total[['RSI']].tail(n=10)


# RSI signal
kor_stock_ohlcv_095570_total['RSI_signal'] = kor_stock_ohlcv_095570_total['RSI'].rolling(window=9).mean()

# MACD 
macd = MACD(close=kor_stock_ohlcv_095570_total['close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
# Stochastic
stoch = StochasticOscillator(high=kor_stock_ohlcv_095570_total['high'],
                             close=kor_stock_ohlcv_095570_total['close'],
                             low=kor_stock_ohlcv_095570_total['low'],
                             window=14, 
                             smooth_window=3)




In [20]:
kor_stock_ohlcv_095570_total['MACD_DIFF'] = macd.macd_diff()
kor_stock_ohlcv_095570_total['MACD'] = macd.macd()
kor_stock_ohlcv_095570_total['MACD_Signal'] = macd.macd_signal()


kor_stock_ohlcv_095570_total['Stoch'] = stoch.stoch()
kor_stock_ohlcv_095570_total['Stoch_Signal'] = stoch.stoch_signal()

In [21]:
kor_stock_ohlcv_095570_total = kor_stock_ohlcv_095570_total[kor_stock_ohlcv_095570_total['date'] > '2023-01-01']


In [47]:
def func1(data):
    fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.4,0.15,0.15,0.15, 0.15])

    # Row 1
    fig.add_trace(go.Candlestick(
        x=data['date'],
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        increasing_line_color= 'red', decreasing_line_color= 'blue', 
        name = '')
    , row=1, col=1)
    
    
    fig.add_trace(go.Scatter(
        x=data['date'],
        y=data['MA5'],
        opacity=0.7,
        line=dict(color='blue', width=2),
        name='MA 5'))
    
    fig.add_trace(go.Scatter(
        x=data['date'],
        y=data['MA20'],
        opacity=0.7,
        line=dict(color='orange', width=2),
        name='MA 20'))
    
    fig.add_trace(go.Scatter(
        x=pd.concat([data['date'], data['date'][::-1]]),
        y=pd.concat([data['upper'], data['lower'][::-1]]),
        fill='toself',
        fillcolor='rgba(255,255,0,0.1)',
        line=dict(color='rgba(255,255,255,0.2)', width=2),
        name='Bollinger Band',
    ))
    
    
    # Row 2
    colors = ['blue' if row['open'] - row['close'] >= 0 
              else 'red' for index, row in data.iterrows()]
    
    fig.add_trace(go.Bar(x=data['date'], 
                         y=data['volume'],
                         marker_color=colors,
                         name = 'Volume'
                        ), row=2, col=1)
    
    
    # Row 3
    colors = ['blue' if val >= 0 
              else 'red' for val in data['MACD_DIFF']]
    fig.add_trace(go.Bar(x=data['date'], 
                         y=data['MACD_DIFF'],
                         marker_color=colors,
                         name =  'MACD DIFF'
                        ), row=3, col=1)
    
    fig.add_trace(go.Scatter(x=data['date'],
                             y=data['MACD'],
                             line=dict(color='green', width=2),
                             name = 'MACD'
                            ), row=3, col=1)
    fig.add_trace(go.Scatter(x=data['date'],
                             y=data['MACD_Signal'],
                             line=dict(color='orange', width=1),
                             name = 'MACD Signal'
                            ), row=3, col=1)
    
    # Row 4
    fig.add_trace(go.Scatter(x=data['date'],
                             y=data['Stoch'],
                             line=dict(color='green', width=1),
                             name = 'Stoch'
                            ), row=4, col=1)
    fig.add_trace(go.Scatter(x=data['date'],
                             y=data['Stoch_Signal'],
                             line=dict(color='orange', width=1),
                             name = 'Stoch Signal'
                            ), row=4, col=1)
    
    # Row 5
    # RSI
    fig.add_trace(go.Scatter(x=data['date'],
                             y=data['RSI'],
                             line=dict(color='red', width=1),
                             name = 'RSI'
                            ), row=5, col=1)
    
    # 수평 사각 영역 추가하기
    
    fig.add_hrect(y0=30, y1=70, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text=" ", 
                  annotation_position="top left",
                  annotation_font_size=20,
                  annotation_font_color="red",
                  annotation_font_family="Times New Roman", row=5, col=1)
    
    fig.add_hrect(y0=40, y1=60, line_width=1, fillcolor="blue", opacity=0.1,
                  annotation_text=" ", 
                  annotation_position="top left",
                  annotation_font_size=20,
                  annotation_font_color="red",
                  annotation_font_family="Times New Roman", row=5, col=1)
        
    # Rayout
    
    fig.update_layout(
        title = '삼성전자 주가',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='black',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_dark',
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y= 1.1), 
        barmode='group',
        margin=go.layout.Margin(
            l=10, #left margin
            r=10, #right margin
            b=10, #bottom margin
            t=50  #top margin
        ),
        height=600, width=800, 
        showlegend=False, 
        xaxis_rangeslider_visible=False
    )
        
    
    
    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
    fig.update_yaxes(title_text="Stoch", row=4, col=1)
    fig.update_yaxes(title_text="RSI", row=5, col=1)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])], row = 1, col = 1)

    return fig


In [49]:
kor_stock_ohlcv_095570_total

,date,open,high,low,close,volume,price_change_percentage,ticker,MA5,MA20,...,하락폭,AU,AD,RSI,RSI_signal,MACD_DIFF,MACD,MACD_Signal,Stoch,Stoch_Signal
1003,2023-01-25,63500,63700,63000,63400,16822710,2.588997,005930,61620.0,59295.0,...,0.0,475.292603,214.046077,68.949069,60.794048,445.444873,840.790193,395.345320,96.296296,96.159196
1004,2023-01-26,63800,63900,63300,63900,13278277,0.788644,005930,62200.0,59595.0,...,0.0,477.057417,198.757071,70.589996,62.148828,499.721643,1019.997374,520.275730,100.000000,96.628680
1005,2023-01-27,64400,65000,63900,64600,18760182,1.095462,005930,63040.0,59920.0,...,0.0,492.981887,184.560138,72.760341,63.744757,547.474147,1204.618414,657.144267,94.366197,96.887498
1006,2023-01-30,64900,64900,63100,63300,20995234,-2.012384,005930,63400.0,60255.0,...,1300.0,457.768895,264.234413,63.402604,64.154285,459.750970,1231.832980,772.082010,68.518519,87.628239
1007,2023-01-31,63500,63700,61000,61000,29746731,-3.633491,005930,63240.0,60540.0,...,2300.0,425.071117,409.646241,50.923958,63.029426,226.847325,1055.641166,828.793841,21.568627,61.484448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,2023-09-26,70000,70000,68400,68600,13143470,-1.152738,005930,69060.0,69940.0,...,800.0,271.979436,341.353839,44.344478,51.668642,-226.555366,65.646522,292.201888,7.500000,15.833333
1172,2023-09-27,68600,69100,68200,68400,14886491,-0.291545,005930,68820.0,70005.0,...,200.0,252.552334,331.257136,43.259376,49.401775,-260.001265,-32.799693,227.201572,4.878049,13.292683
1173,2023-10-04,67400,67700,66700,67500,23361149,-1.315789,005930,68540.0,70035.0,...,900.0,234.512881,371.881626,38.673319,46.481259,-326.842061,-181.351005,145.491056,14.285714,8.887921
1174,2023-10-05,67300,67400,66700,66700,16108313,-1.185185,005930,68120.0,69820.0,...,800.0,217.761961,402.461510,35.110242,44.517274,-403.983522,-359.488346,44.495176,0.000000,6.387921


In [48]:

func1(kor_stock_ohlcv_095570_total)

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

